In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install torch
!pip install pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.9 MB/s eta 0:00:00


In [ ]:
from PIL import Image
im = Image.open("/content/drive/MyDrive/Champion-1.jpg")
im.show()

In [6]:

from transformers import pipeline

image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

text = image_to_text("/content/drive/MyDrive/Champion-1.jpg")


Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.1.attn.masked_bias', 'decoder.transformer.h.11.crossattention.masked_bias', 'decoder.transformer.h.7.attn.bias', 'decoder.transformer.h.7.attn.masked_bias', 'decoder.transformer.h.10.crossattention.masked_bias', 'decoder.transformer.h.3.crossattention.bias', 'decoder.transformer.h.3.attn.masked_bias', 'decoder.transformer.h.4.crossattention.bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.11.attn.bias', 'decoder.transformer.h.11.attn.masked_bias', 'decoder.transformer.h.6.attn.bias', 'decoder.transformer.h.5.attn.masked_bias', 'decoder.transformer.h.10.attn.bias', 'decoder.transformer.h.9.crossattention.bias', 'decoder.transformer.h.2.crossattention.bias', 'decoder.transformer.h.2.crossattention.masked_bias', 'decoder.transformer.h.0.crossattention.masked_bias', 'decoder.transformer.h.0.crossattention.bi

In [7]:
text

[{'generated_text': 'a soccer game with a team of young boys playing '}]

In [8]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds



Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.1.attn.masked_bias', 'decoder.transformer.h.11.crossattention.masked_bias', 'decoder.transformer.h.7.attn.bias', 'decoder.transformer.h.7.attn.masked_bias', 'decoder.transformer.h.10.crossattention.masked_bias', 'decoder.transformer.h.3.crossattention.bias', 'decoder.transformer.h.3.attn.masked_bias', 'decoder.transformer.h.4.crossattention.bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.11.attn.bias', 'decoder.transformer.h.11.attn.masked_bias', 'decoder.transformer.h.6.attn.bias', 'decoder.transformer.h.5.attn.masked_bias', 'decoder.transformer.h.10.attn.bias', 'decoder.transformer.h.9.crossattention.bias', 'decoder.transformer.h.2.crossattention.bias', 'decoder.transformer.h.2.crossattention.masked_bias', 'decoder.transformer.h.0.crossattention.masked_bias', 'decoder.transformer.h.0.crossattention.bi

In [9]:
text = predict_step(['/content/drive/MyDrive/Champion-1.jpg'])

In [10]:
text

['soccer players are playing soccer on a field']